In [1]:
import pandas as pd
import numpy as np


pd.options.display.max_rows = 60
pd.options.display.max_columns = 70

In [2]:
h1_res = pd.read_pickle("pickle/h1_res.pick")
h2_res = pd.read_pickle("pickle/h2_res.pick")
h1_dbd = pd.read_pickle("pickle/h1_dbd.pick")
h2_dbd = pd.read_pickle("pickle/h2_dbd.pick")

In [3]:
h1_dbd.columns

Index(['DOW', 'Occ', 'RoomsSold', 'ADR', 'RoomRev', 'RevPAR', 'NumCancels',
       'Trn_RoomsSold', 'Trn_ADR', 'Trn_RoomRev', 'Grp_RoomsSold', 'Grp_ADR',
       'Grp_RoomRev', 'TrnP_RoomsSold', 'TrnP_ADR', 'TrnP_RoomRev',
       'Cnt_RoomsSold', 'Cnt_ADR', 'Cnt_RoomRev', 'WE', 'WD', 'STLY_Date'],
      dtype='object')

In [24]:
h2_dbd.index = pd.to_datetime(h2_dbd.index, format="%Y-%m-%d")

In [135]:
h1_pricing = h1_dbd[["Trn_RoomsSold", "Trn_RoomRev", "WD"]].groupby([pd.Grouper(freq='1W'), 'WD']).agg('sum')
# h1_pricing = h1_pricing.reset_index().rename(columns={'level_0': 'Date'})
# h1_pricing['Date'] = pd.to_datetime(h1_pricing.Date, format='%Y-%m-%d')
h1_pricing["Trn_ADR"] = round(h1_pricing.Trn_RoomRev / h1_pricing.Trn_RoomsSold, 2)
h1_pricing.head(10)

Trn_RoomsSold  Trn_RoomRev  Trn_ADR
           WD                                        
2015-07-05 False          125.0     13567.03   108.54
           True           152.0     15863.32   104.36
2015-07-12 False          236.0     27208.52   115.29
           True           533.0     60669.98   113.83
2015-07-19 False          262.0     36796.28   140.44
           True           614.0     80064.80   130.40
2015-07-26 False          303.0     44216.85   145.93
           True           757.0    106470.32   140.65
2015-08-02 False          285.0     43091.88   151.20
           True           739.0    107954.47   146.08

In [76]:
h1_pricing.loc[0, 'Date'].weekday()

6

In [78]:
h2_pricing = h2_dbd[["Trn_RoomsSold", "Trn_RoomRev", "WD"]].groupby([pd.Grouper(freq='1W'), 'WD']).agg('sum')
h2_pricing = h2_pricing.reset_index().rename(columns={'level_0': 'Date'})
h2_pricing['Date'] = pd.to_datetime(h2_pricing.Date, format='%Y-%m-%d')
h2_pricing.index=h2_pricing.Date
h2_pricing["Trn_ADR"] = round(h2_pricing.Trn_RoomRev / h2_pricing.Trn_RoomsSold, 2)
h2_pricing.head(10)

,Date,WD,Trn_RoomsSold,Trn_RoomRev,Trn_ADR
Date,,,,,
2015-07-05,2015-07-05,False,6.0,496.54,82.76
2015-07-05,2015-07-05,True,133.0,12608.27,94.80
2015-07-12,2015-07-12,False,15.0,1017.78,67.85
2015-07-12,2015-07-12,True,23.0,1535.72,66.77
2015-07-19,2015-07-19,False,3.0,62.00,20.67
2015-07-19,2015-07-19,True,15.0,817.45,54.50
2015-07-26,2015-07-26,False,13.0,247.00,19.00
2015-07-26,2015-07-26,True,18.0,1007.74,55.99
2015-08-02,2015-08-02,False,26.0,1722.26,66.24


In [64]:
# now we need to pull these into DF_DBDs

In [132]:
import datetime
def map_prices(date):
    date = datetime.date(2015,7,19)
    date_str = datetime.datetime.strftime(date,format='%Y-%m-%d')
    WD = date.weekday() not in (4, 5)
    mask = (h1_pricing.WD==WD) & (h1_pricing.Date == date_str)
    price = int(h1_pricing[mask]['Trn_ADR'])

    return price

In [131]:

date = datetime.date(2015,7,19)
date_str = datetime.datetime.strftime(date,format='%Y-%m-%d')
WD = date.weekday() not in (4, 5)
mask = (h1_pricing.WD==WD) & (h1_pricing.Date == date_str)
price = int(h1_pricing[mask]['Trn_ADR'])
price


130

In [134]:
h1_dbd["Date"] = h1_dbd.index
h1_dbd["Price"] = h1_dbd.Date.map(map_prices)
h1_dbd.head(20)

,DOW,Occ,RoomsSold,ADR,RoomRev,RevPAR,NumCancels,Trn_RoomsSold,Trn_ADR,Trn_RoomRev,Grp_RoomsSold,Grp_ADR,Grp_RoomRev,TrnP_RoomsSold,TrnP_ADR,TrnP_RoomRev,Cnt_RoomsSold,Cnt_ADR,Cnt_RoomRev,WE,WD,STLY_Date,Date,Price
2015-07-01,Wed,0.19,36.0,93.25,3356.83,17.95,5.0,30.0,95.91,2877.25,0.0,0.00,0.00,0.0,0.00,0.00,6.0,79.93,479.58,False,True,2014-07-02,2015-07-01,130
2015-07-02,Thu,0.34,64.0,97.68,6251.33,33.43,14.0,43.0,107.71,4631.34,0.0,0.00,0.00,2.0,75.23,150.46,19.0,77.34,1469.53,False,True,2014-07-03,2015-07-02,130
2015-07-03,Fri,0.43,81.0,99.41,8052.35,43.06,27.0,54.0,109.70,5923.95,0.0,0.00,0.00,2.0,75.23,150.46,25.0,79.12,1977.94,True,False,2014-07-04,2015-07-03,130
2015-07-04,Sat,0.58,108.0,96.98,10473.47,56.01,39.0,71.0,107.65,7643.08,0.0,0.00,0.00,5.0,74.37,371.86,32.0,76.83,2458.53,True,False,2014-07-05,2015-07-04,130
2015-07-05,Sun,0.65,122.0,97.56,11901.72,63.65,32.0,79.0,105.76,8354.73,1.0,153.00,153.00,5.0,74.37,371.86,37.0,81.68,3022.13,False,True,2014-07-06,2015-07-05,130
2015-07-06,Mon,0.76,142.0,101.05,14349.04,76.73,30.0,96.0,110.34,10592.48,0.0,0.00,0.00,3.0,83.75,251.26,43.0,81.52,3505.30,False,True,2014-07-07,2015-07-06,130
2015-07-07,Tue,0.82,153.0,103.08,15771.44,84.34,37.0,103.0,112.27,11564.28,0.0,0.00,0.00,3.0,83.75,251.26,47.0,84.17,3955.90,False,True,2014-07-08,2015-07-07,130
2015-07-08,Wed,0.84,157.0,104.77,16448.71,87.96,44.0,108.0,112.77,12178.80,0.0,0.00,0.00,5.0,96.29,481.46,44.0,86.10,3788.45,False,True,2014-07-09,2015-07-08,130
2015-07-09,Thu,0.90,168.0,105.89,17790.31,95.14,45.0,114.0,113.10,12893.40,0.0,0.00,0.00,4.0,136.38,545.54,50.0,87.03,4351.37,False,True,2014-07-10,2015-07-09,130
2015-07-10,Fri,0.95,177.0,108.87,19269.19,103.04,45.0,120.0,114.33,13719.88,0.0,0.00,0.00,6.0,156.59,939.54,51.0,90.39,4609.77,True,False,2014-07-11,2015-07-10,130


In [49]:
h1_dbd.head(3)

,DOW,Occ,RoomsSold,ADR,RoomRev,RevPAR,NumCancels,Trn_RoomsSold,Trn_ADR,Trn_RoomRev,Grp_RoomsSold,Grp_ADR,Grp_RoomRev,TrnP_RoomsSold,TrnP_ADR,TrnP_RoomRev,Cnt_RoomsSold,Cnt_ADR,Cnt_RoomRev,WE,WD,STLY_Date
2015-07-01,Wed,0.19,36.0,93.25,3356.83,17.95,5.0,30.0,95.91,2877.25,0.0,0.0,0.0,0.0,0.00,0.00,6.0,79.93,479.58,False,True,2014-07-02
2015-07-02,Thu,0.34,64.0,97.68,6251.33,33.43,14.0,43.0,107.71,4631.34,0.0,0.0,0.0,2.0,75.23,150.46,19.0,77.34,1469.53,False,True,2014-07-03
2015-07-03,Fri,0.43,81.0,99.41,8052.35,43.06,27.0,54.0,109.70,5923.95,0.0,0.0,0.0,2.0,75.23,150.46,25.0,79.12,1977.94,True,False,2014-07-04


In [21]:
h1_dbd.index

DatetimeIndex(['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04',
               '2015-07-05', '2015-07-06', '2015-07-07', '2015-07-08',
               '2015-07-09', '2015-07-10',
               ...
               '2017-08-22', '2017-08-23', '2017-08-24', '2017-08-25',
               '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29',
               '2017-08-30', '2017-08-31'],
              dtype='datetime64[ns]', length=793, freq=None)

In [25]:
h2_dbd.index

DatetimeIndex(['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-04',
               '2015-07-05', '2015-07-06', '2015-07-07', '2015-07-08',
               '2015-07-09', '2015-07-10',
               ...
               '2017-08-22', '2017-08-23', '2017-08-24', '2017-08-25',
               '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29',
               '2017-08-30', '2017-08-31'],
              dtype='datetime64[ns]', length=793, freq=None)

In [5]:
h1_dbd[["Trn_RoomsSold", "Trn_ADR"]]

,Trn_RoomsSold,Trn_ADR
2015-07-01,30.0,95.91
2015-07-02,43.0,107.71
2015-07-03,54.0,109.70
2015-07-04,71.0,107.65
2015-07-05,79.0,105.76
...,...,...
2017-08-27,125.0,211.64
2017-08-28,130.0,205.76
2017-08-29,130.0,198.38
2017-08-30,130.0,187.56
